In [ ]:
import subprocess
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import seaborn as sns
from itertools import cycle
import colorsys
import re
from matplotlib.ticker import PercentFormatter
from random import shuffle

In [ ]:
with open("all_test_results.pkl", "rb") as f:
    all_test_results: pd.DataFrame = pickle.load(f)

In [ ]:
grouped_n = 2
all_test_results["trainable_params_grouped"] = grouped_n ** np.round(np.log(all_test_results["trainable_params"]) / np.log(grouped_n))

In [ ]:
def min_max(x):
  return x.min(), x.max()

group_keys = ["dataset", "type", "model", "trainable_params_grouped"]

stats = (
    all_test_results
    .groupby(group_keys, as_index=False)
    .agg(min=("test_acc", "min"),
         max=("test_acc", "max"),
         test_acc=("test_acc", "mean"),
         std=("test_acc", "std"))
)

max_points = (
    all_test_results
    .loc[all_test_results.groupby(group_keys[:-1])["test_acc"].idxmax()]
    .assign(max_type="max_point")
)

max_mean_points = (
    stats
    .loc[stats.groupby(group_keys[:-1])["test_acc"].idxmax()]
    .assign(max_type="max_mean_point")
)

def to_percent(key: str):
  def f(x: pd.Series):
    return x[key].map("{:.2%}".format)
  return f

max_values = (
  pd.concat([max_points, max_mean_points])
    .set_index(["dataset", "type", "max_type"])
    .sort_values("test_acc", ascending=False)
    .sort_index(level=["dataset", "type", "max_type"])
    .assign(test_acc_percent=to_percent("test_acc"), std_percent=to_percent("std"))
)

stats.set_index(["dataset", "type"], inplace=True)

In [ ]:
all_test_results

In [ ]:
all_test_results[(all_test_results[["dataset", "type"]] == ("cifar10", "dropout")).all(axis=1)]

In [ ]:
from opt import get_model_idx
get_model_idx("dend_ann_local_rfs")

In [ ]:
out = ""
for dt, data in all_test_results.groupby(["dataset", "type"]):
    new_data = {}
    for k, v in data.groupby(["model", "num_dends", "num_soma", "sigma", "num_epochs_min"]):
        if len(v) == 5:
            d = dict(
                acc_mean=v["test_acc"].mean(),
                acc_std=v["test_acc"].std(),
                loss_mean=v["test_loss"].mean(),
                loss_std=v["test_loss"].std()
            )
            new_data[k] = d
    print(*dt, sep=" - ")
    new_data = (
        pd.DataFrame(new_data).T
        .sort_values("acc_mean", ascending=False)
        .reset_index(names=["model", "num_dends", "num_soma", "sigma", "num_epochs_min"])
    )
    display(new_data[:10])
    for i, row in new_data[:10].iterrows():
        if "dropout" in row["model"]:
            model = "_".join(row["model"].split("_")[:-2])
        else:
            model = row["model"]
        out += f"seq 5 | xargs -I [] uv run main.py --trial [] --model {get_model_idx(model)} --dataset {dt[0]} --num-layers 2 --sigma {row['sigma']} -d {row['num_dends']} -s {row['num_soma']} -o best_test --backend tensorflow &\n"

print(out)


In [ ]:

for dt, data in all_test_results.groupby(["dataset", "type"]):
  keys = all_test_results["model"].unique()
  colors = dict(zip(keys, sns.color_palette("husl", len(keys))))

  if dt[1] == "other_fmnist":
    continue

  fig, ax = plt.subplots()

  sns.lineplot(data, x="trainable_params_grouped", y="test_acc", hue="model", errorbar="sd", palette=colors)
  sns.lineplot(stats.loc[dt], x="trainable_params_grouped", y="min", hue="model", linestyle="--", alpha=0.25, legend=False, palette=colors)
  sns.lineplot(stats.loc[dt], x="trainable_params_grouped", y="max", hue="model", linestyle="--", alpha=0.25, legend=False, palette=colors)
  sns.scatterplot(data=max_values.loc[*dt, "max_point"], x="trainable_params", y="test_acc", hue="model", palette=colors, legend=False, marker="X")
  sns.scatterplot(data=max_values.loc[*dt, "max_mean_point"], x="trainable_params_grouped", y="test_acc", hue="model", palette=colors, legend=False, marker="*", s=125)
  
  plt.xscale("log")
  ax.yaxis.set_major_formatter(PercentFormatter(xmax=1.0))
  plt.title('\n'.join(dt))
  
  plt.show()

  print(
    " - ".join(dt), "",
    "max points:", 
    max_values[["model","test_acc_percent", "trainable_params"]]
      .loc[(*dt, "max_point")]
      .reset_index(drop=True)
      .to_string(),
    "\nmax mean points:", 
    max_values[["model", "test_acc_percent", "trainable_params_grouped", "std_percent"]]
      .loc[(*dt, "max_mean_point")]
      .reset_index(drop=True)
      .to_string(),
    sep='\n')

In [ ]:
data1 = all_test_results[all_test_results["file_name"] == "fmnist_1_layer_noise.png"]
# TODO: don't forget to look at the noise graph ^ low priority but WIP.

In [ ]:
from utils import num_trainable_params
from opt import make_masks, get_ordered_model_names
from main import update_model_config
from collections import namedtuple

# for i in range(2, 5):
#     for j in range(2, 5):
# print(i,j)
i, j = 1, 2
num_dendrites = 4**i
num_somas = 4**j
# print(2**i,2**j)
num_layers = 2
synapses = 16 # doesn't do shit
img_width = 28
img_height = 28
num_classes = 10
channels = 1
seed = 7

dends = [num_dendrites]*num_layers
somas = [num_somas]*num_layers

m = namedtuple("modelConfig", ["conventional", "rfs", "model", "all_to_all", "sparse"])
m.conventional = False
m.rfs = None
m.all_to_all = False
m.sparse = False
m.model = 1

update_model_config(m)

name = get_ordered_model_names()[m.model]
print(name)

masks = make_masks(
    dends, somas, synapses, num_layers, img_width, img_height, num_classes, channels, m.conventional, m.rfs is not None, m.rfs, name, seed
)

shift = 1
new_order = [3-(x-shift)%4 for x in range(4)]
axes = [m//2 for m in new_order]
# reshaped_masks = [
#     m.reshape(
#         [int(np.sqrt(m.shape[i])) for i in axes]
#     ).transpose(new_order)
#     if m.ndim == 2 else 
#     m.reshape(
#         int(np.sqrt(m.shape[0])), 
#         int(np.sqrt(m.shape[0]))
#     ) for m in masks[:-2] 
# ]
t2 = [(x.shape, x.sum().tolist()) for x in masks]
t3 = [int(np.prod(x)) for x, _ in t2]
t4 = [x+y for x, y in zip(t3[::2], t3[1::2])]

# print([y for x, y in t2])
# tt = [0,4**(j+i),4**(j+i),4**j]*num_layers+[4**j*num_classes, num_classes, 0]
# print(tt)
# tt = (4**(j+i),4**j,)*2
# print([tt[s:e] for s,e in [(0,2),(0,4),(2,4)]])
print(*t2, sep='\n')
# print([y for x, y in t2])
# print(t3)
# print(t4)
# print(sum(tt))
# print(sum(mask.sum() for mask in masks))
print(num_trainable_params(num_dendrites, num_somas, img_width*img_height*channels, synapses, num_classes, num_layers, name))

In [ ]:
for i, t in enumerate(masks):
    count = 0
    if t.ndim == 1:
        plt.imshow(t[:,np.newaxis])
    else:
        plt.imshow(t)
    plt.show()

In [ ]:
for i, t in enumerate(reshaped_masks):
    count = 0
    if i % 4 == 0:
        continue
    if i % 2 == 0:
        for i in range(t.shape[2]):
            for j in range(t.shape[3]):
                # count += (t[:,:,i,j].any())
                plt.imshow(t[:,:,i,j])
                plt.show()
    else:
        plt.imshow(t)
        plt.show()
    print(count)

In [ ]:
Model: "dend_ann_random"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dend_1 (Dense)                       │ (None, 64)                  │          50,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dend_1_relu (ReLU)                   │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ soma_1 (Dense)                       │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ soma_1_relu (ReLU)                   │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dend_2 (Dense)                       │ (None, 64)                  │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dend_2_relu (ReLU)                   │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ soma_2 (Dense)                       │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ soma_2_relu (ReLU)                   │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │             170 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

In [ ]:
import os
import sys
import keras
from masked_dense import MaskedDense

p = "all_out_2/results_mnist_2_layer/"
models = {}
m = namedtuple("modelConfig", ["conventional", "rfs", "model", "all_to_all", "sparse"])

for d in os.listdir(p):
    ms = os.listdir(p+d)
    models.setdefault(d, {})
    models[d]["untrained"] = keras.models.load_model(p+d+"/"+ms[0], custom_objects={'MaskedDense': MaskedDense})
    models[d]["trained"] = keras.models.load_model(p+d+"/"+ms[1], custom_objects={'MaskedDense': MaskedDense})

from typing import Callable, Iterable
def pp(x: Iterable, n: int | list[int]):
    if isinstance(n, int):
        print(' | '.join([f"{k}{' '*(n-len(str(k)))}" for k in x]))
    else:
        print(' | '.join([f"{k}{' '*(m-len(str(k)))}" for m, k in zip(n,x)]))

for i in range(12):
    m.conventional = False
    m.rfs = None
    m.all_to_all = False
    m.sparse = False
    m.model = i
    update_model_config(m)
    name = get_ordered_model_names()[i]
    masks = make_masks(
        dends, somas, synapses, num_layers, img_width, img_height, num_classes, channels, m.conventional, m.rfs is not None, m.rfs, name, seed
    )

    W1 = models[name]["untrained"].weights
    W2 = models[name]["trained"].weights
    # t2 = [f"shape={x.shape}; params={x.sum().tolist()}" for x in masks]
    # print([int((abs(w2-w1)>0).numpy().sum()) for x, w1, w2 in zip(W1, W2)])
    pp([name]+[f"{int(x.sum())} vs {int((w1!=w2).numpy().sum())}" for x, w1, w2 in zip(masks, W1, W2)], [22,14,8,12,8,12,8,12,8,11,5])
    # print((np.abs(w1-w2) > 0).sum())

pp(["shapes"]+[x.shape for x in masks], [22,14,8,12,8,12,8,12,8,11,5])

In [ ]:
w1 = models[get_ordered_model_names()[1]]["untrained"].weights
w2 = models[get_ordered_model_names()[1]]["trained"].weights

print((abs(w1[0]-w2[0])>0).numpy().sum())
print((abs(w2[1]-w1[1])>0).numpy().sum())
print((abs(w1[2]-w2[2])>0).numpy().sum())
print((abs(w1[4]-w2[4])>0).numpy().sum())
print((abs(w1[6]-w2[6])>0).numpy().sum())

In [ ]:
from receptive_fields import *
from matplotlib import pyplot as plt
import seaborn as sns
def make_masks_local(
    dends, soma, synapses, num_layers, img_width, img_height,
    num_classes=10, channels=1, conventional=False, sparse=False,
    rfs=True, rfs_type='somatic', rfs_mode='random',
    input_sample=None, seed=None, *, np=np
    ):
    """
    Make masks to transform a traditional ANN in a dendritic ANN.

    Parameters
    ----------
    dends : list
        Number of dendrites/soma per layer.
    soma : list
        Number of somata per layer.
    num_layers : int
        Number of dendrosomatic layers.
    img_width : int
        The width of the input images.
    img_height : int
        The height of the input images.
    channels : int
        The number of channels of the input images.
    conventional : boolean
        If the model is of all-to-all type (True) or not (False).
        Default is False.
    sparse : boolean
        If the model is of random (True) or structured (False) sparse
        connections. Default is False.
    rfs : boolean
        If the model is of RFs (True) or random (False) structured connections.
        Default is True.
    rfs_type : str
        Type of RFs; local (`dendritic`) or global (`somatic`).
        Default is `somatic`.
    rfs_mode : str
        Mode of rfs construction. Default is `random`. Other valid options
        are `one_to_one` and `constant`. Refer to receptive_fields.py in
        random_connectivity function for more information.
    np : Callable, optional
        Numpy like backend (e.g. numpy vs jax.numpy)

    Returns
    -------
    masks : list
        A list with np.ndarrays containing the boolean masks for all layer
        weights and biases.

    """
    masks = []
    for i in range(num_layers):
        if i == 0:
            # first layer --> create a matrix with input dimensions.
            matrix = np.zeros((img_width, img_height))
        else:
            # for the rest dendrosomatic layers the input is a `square` form of
            # the previous layer's somata.
            divisors = [j for j in range(1, soma[i-1] + 1) if soma[i-1] % j == 0]
            ix = len(divisors) // 2
            if len(divisors) % 2 == 0:
                matrix = np.zeros((divisors[ix], divisors[ix - 1]))
            else:
                matrix = np.zeros((divisors[ix], divisors[ix]))

        # when RFs are enabled!
        print(rfs is not None, rfs_mode)
        if rfs:
            print("shape:", matrix.shape)
            mask_s_d, centers = receptive_fields(
                matrix, somata=soma[i],
                dendrites=dends[i],
                num_of_synapses=synapses,
                opt=rfs_mode,
                rfs_type=rfs_type,
                prob=0.7,
                num_channels=channels if i == 0 else 1,
                seed=seed
            )
            c = list(zip(*centers))
            sns.scatterplot(x=c[0], y=c[1])
            plt.show()
            plt.imshow(mask_s_d)
            plt.show()
        else:
            # if no RFs are enabled use random connectivity (like `sparse`)
            inputs_size = matrix.size
            factor = channels if i == 0 else 1

            # for soma to the next dendrites (if more than two layers)
            mask_s_d = random_connectivity(
                inputs=inputs_size*factor,
                outputs=soma[i]*dends[i],
                conns=synapses*soma[i]*dends[i],
                seed=seed,
            )
            plt.imshow(mask_s_d)
            plt.show()
        masks.append(mask_s_d)
        # create a mask with `ones` for biases
        masks.append(np.ones((mask_s_d.shape[1], )).astype('int'))
        # Create structured connectivity if not `sparse`,
        # else random (i.e., sparse).
        if not sparse:
            mask_d_s = connectivity(
                inputs=dends[i]*soma[i],
                outputs=soma[i]
            )
        else:
            mask_d_s = random_connectivity(
                inputs=dends[i]*soma[i],
                outputs=soma[i],
                conns=dends[i]*soma[i],
                seed=seed,
            )
        plt.imshow(mask_d_s)
        plt.show()
        # Append the masks
        masks.append(mask_d_s)
        # create a mask with `ones` for biases
        masks.append(np.ones((mask_d_s.shape[1], )).astype('int'))

    # If vanilla ANN --> re-write the masks with ones
    # for vanilla ANN all-to-all connectivity and RFs
    if conventional:
        if rfs or sparse:
            # vanilla ANN with random, sparse inputs, or RFs
            for i, m in enumerate(masks):
                # `4` denotes the number of masks per dendrosomatic layer
                # So, elements 0, 4, 8, 12, etc will have the masks defined above.
                # All other layers will have masks filled with ones.
                if i % 4 != 0:
                    masks[i] = np.ones_like(m)
        else:
            # vanilla ANN; create all masks with `ones`
            for i, m in enumerate(masks):
                masks[i] = np.ones_like(m)

    # dendritic or sparse all-to-all
    if input_sample == 'all_to_all':
        for i, m in enumerate(masks):
            # `4` denotes the number of masks per dendrosomatic layer
            # So, elements 0, 4, 8, 12, etc will take masks filled with ones.
            if i % 4 == 0:
                masks[i] = np.ones_like(m)

    # Add two masks for the output layer (weights and biases) set to 1.
    masks.append(np.ones((masks[-2].shape[1], num_classes)).astype('int'))
    masks.append(np.ones((num_classes, )).astype('int'))

    return masks

In [ ]:
from collections import namedtuple
from main import update_model_config
from opt import get_ordered_model_names
i, j = 1, 2
num_dendrites = 4**i
num_somas = 4**j
# print(2**i,2**j)
num_layers = 2
synapses = 16 # doesn't do shit
img_width = 28
img_height = 28
num_classes = 10
channels = 1
seed = 1

dends = [num_dendrites]*num_layers
somas = [num_somas]*num_layers


m = namedtuple("modelConfig", ["conventional", "rfs", "model", "all_to_all", "sparse"])
m.conventional = False
m.rfs = None
m.all_to_all = False
m.sparse = False
m.model = 1

update_model_config(m)

name = get_ordered_model_names()[m.model]
print(name)
print(m.rfs)

masks = make_masks_local(
    dends, somas, synapses, num_layers, img_width, img_height, 
    num_classes, channels, m.conventional, m.sparse, m.rfs is not None, m.rfs, seed=seed
)


In [ ]:
for i in range(12):
    m.conventional = False
    m.rfs = None
    m.all_to_all = False
    m.sparse = False
    m.model = i
    update_model_config(m)
    name = get_ordered_model_names()[i]
    masks = make_masks(
        dends, somas, synapses, num_layers, img_width, img_height, num_classes, channels, m.conventional, m.rfs is not None, m.rfs, name, seed
    )
